In [55]:
using Pkg; Pkg.activate("../.")
using Revise
using QuadrupedBalance
using TOML
using RobotDynamics
using LinearAlgebra
using OSQP
using SparseArrays
using ControlSystems
using RigidBodyDynamics
using Rotations
using PyPlot
using MeshCat
using MeshCatMechanisms
using StaticArrays
using DelimitedFiles
const QB = QuadrupedBalance
const RD = RobotDynamics

  Activating project at `~/dev/QuadrupedBalance.jl`


RobotDynamics

In [57]:
urdfpath = joinpath(@__DIR__, "..", "src","a1_rw","urdf","a1_rw.urdf")
A1mech = parse_urdf(urdfpath, floating=true, remove_fixed_tree_joints=true);

model_full = QB.UnitreeA1FlyWheel(A1mech);

In [318]:
#### Simulation and helper Functions 
"""Take a single semi-implicit euler integration step. Returns x_next and λ"""
function semi_implicit_euler(A1::QB.AbstractQuadruped, x::AbstractVector, u::AbstractVector, ϕ_cons::AbstractVector, foot_indices, h)
    xn = copy(x)
    v_dim = num_velocities(A1.rigidbody.mech)
    p_dim = num_positions(A1.rigidbody.mech)
    
    # Mass matrix and dynamics bias 
    M = QB.get_mass_matrix(A1, xn)
    C_dyn = QB.get_dynamics_bias(A1, xn)
    
    # Kinematic Jacobian 
    attitude_error_jacobian = blockdiag(0.5*sparse(QuadrupedBalance.quaternion_differential(x[1:4])),
                                        sparse(Rotations.UnitQuaternion(x[1:4])),
                                        sparse(I(34)) )
    
    J = QB.dfk_world(x)[foot_indices,:] * attitude_error_jacobian
    J = J[:, 1:v_dim]
    ϕ = QB.fk_world(xn)[foot_indices]


    ## Solving for velocities 
    damp = 0.5
    u_body = [0,0,0, ([1,0,0] * -u[5] + [0,1,0] * -u[6])... ]
    r = [([u_body;u].-C_dyn - [zeros(6);damp*xn[p_dim+6+1:end]])*h.+M*xn[p_dim+1:end]; -ϕ+ϕ_cons] # KKT conditions 
    H = zeros(v_dim+length(foot_indices), v_dim+length(foot_indices))  
    H[1:v_dim,1:v_dim] .= M 
    H[1:v_dim,v_dim+1:end] .= J'*h 
    H[v_dim+1:end, 1:v_dim] .= J*h
    δ = H \ r

    v_next = δ[1:v_dim]
    λ = δ[v_dim+1:end]

    ## Integrating 
    rot = UnitQuaternion(xn[1:4])
    v_trans = rot * v_next[4:6]   
    xn[5:7] = xn[5:7] + v_trans * h
    xn[8:p_dim] .= xn[8:p_dim] + v_next[7:end] *h


    # Update attitude
    q_dot = 0.5*QB.get_G(x[1:4])*v_next[1:3]*h
    xn[1:4] .= QB.L(xn[1:4]) * QB.ρ(v_next[1:3]*h)
    xn[1:4] .= xn[1:4]/norm(x[1:4])
    xn[p_dim+1:end] .= v_next 
    return xn, λ
end 

function actuator_indices(model::QB.UnitreeA1FlyWheel)
    return [8:11; 14:21], 12:13
end 

# Overwriting 
function QB.fk_world(x::AbstractVector)
    leg_inds = [8:11; 14:21]
    q = x[leg_inds]
    quat = Rotations.UnitQuaternion(x[1:4])
    pos = x[5:7]

    p_body = QB.fk(q)
    p_world = copy(p_body)

    for i in 1:4 
        p_world[(i-1)*3 .+ (1:3)] = quat * p_body[(i-1)*3 .+ (1:3)] + pos 
    end 
    
    return p_world 
    
end 

function joint_pd_control(q, q_des, q_v, Kp, Kd)
    q_diff = q - q_des 
    u = -Kp .* q_diff - Kd .* q_v 
end 

joint_pd_control (generic function with 1 method)

In [325]:
let 
    # Specifying what foot is in contact
    foot_contacts = [1, 0, 0, 1] # FR, FL, RR, RL
    foot_indices = []            # specifies which foot constraints are active 
    for i in 1:length(foot_contacts)
        if(foot_contacts[i] == 1)
            append!(foot_indices, (i-1)*3 .+ (1:3))
        end 
    end 
    
    # Findinng the correct joint angles for desired foot hold position using inv kinematics
    z_des = 0.25
    # FR, FL, RR, RL
    p = [0.182, -0.132, -z_des,
         0.182, 0.132, -z_des, 
         -0.183, -0.132, -z_des, 
         -0.18, 0.132, -z_des]
    
    q_guess = [0, 0, 0, 0,
               0.5, 0.5, 0.5, 0.5,
               -1.0, -1.0, -1.0, -1.0]
    
    q = QB.inv_kin(p, q_guess)
    
    ind_legs, ind_w = actuator_indices(model_full)
    x = zeros(41); x[1] = 1.0; x[7] = z_des
    x[ind_legs] = q 
    x[ind_w] = [0.0, 0.5]
    
    
    
    ϕ_world = QB.fk_world(x)[foot_indices]
    u = zeros(14)
        
    h = 0.001
    tf = 4.0
    times = 0:h:tf
    xs = zeros(length(times), 41)
    xs[1, :] = x
    λs = zeros(length(times), 12)
#     u[ind_w] = [1.0,1.0]
#     u[6] = 20.0
#     u[5] = -100
    
    for i in 2:length(times) 
        joint_pos = xs[i-1,ind_legs]
        joint_v = xs[i-1, 20 .+ ind_legs]
        u_joint = joint_pd_control(joint_pos ,q, joint_v, 200, 5)
        u[ind_legs .- 7] = u_joint
        
        quat_now = xs[i-1,1:4]
        quat_diff = UnitQuaternion(quat_now)' * UnitQuaternion(1,0,0,0)
        diff = Rotations.params(quat_diff)
        u[5] = diff[2] * 100
        u[6] = -diff[3] * 1000
        xs[i,:], λ = semi_implicit_euler(model_full, xs[i-1,:], u, ϕ_world, foot_indices, 0.001)
    end
    
    q_anim = [xs[i,1:21] for i in 1:length(times)-1]
    animation = Animation(mvis, times[1:50:end-1], q_anim[1:50:end])
    setanimation!(mvis, animation)
end  

In [10]:
vis = Visualizer() 
cur_path = pwd()
cd(joinpath(@__DIR__,"..","src", "a1_rw", "urdf"))
mvis = MechanismVisualizer(A1mech, URDFVisuals(urdfpath), vis)
cd(cur_path)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/chiyen/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


In [162]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x77, 0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("trunk" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x62, 0x61, 0x73, 0x65, 0x2f, 0x74, 0x72, 0x75, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_floating_base" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x61, 0x74, 0x69, 0x6e, 0x67, 0x5f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_hip" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x6e, 0x6b, 0x2f, 0x52, 0x4c, 0x5f, 0x68, 0x69, 0x70], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_thigh" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x70, 0x2f, 0x52, 0x4c, 0x5f, 0x74, 0x68, 0x69, 0x67, 0x68], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_thigh_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x69, 0x67, 0x68, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_17" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x37], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "RL_calf" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x68, 0x2f, 0x52, 0x4c, 0x5f, 0x63, 0x61, 0x6c, 0x66], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("RL_foot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x66, 0x2f, 0x52, 0x4c, 0x5f, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_RL_foot_fixed" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6f, 0x6f, 0x74, 0x5f, 0x66, 0x69, 0x78, 0x65, 0x64], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_19" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31, 0x39], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72,